In [25]:
#WIKIPEDIA BASED TOOLS:-1
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=5,doc_content_char_limit=500)
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper)

In [26]:
#ARXIV BASED TOOLS:-2
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
arxiv_wrapper=ArxivAPIWrapper(top_k_results=5,doc_content_char_limit=500)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [27]:
#WEBBASED LOADER TOOLS:-3
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.tools.retriever import create_retriever_tool

loader=WebBaseLoader("https://modelcontextprotocol.io/docs/getting-started/intro")
document=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(document)
db=Chroma(collection_name="MultiRag",embedding_function=HuggingFaceEmbeddings())
doc_id=db.add_documents(text_splitter)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":3})
web_tool = create_retriever_tool(
    retriever,
    "model context protocol",
    "Search for information about Model Context Protocol. For any questions about Model Context Protocol, you must use this tool!",
)

In [28]:
Tools=[wikipedia,arxiv,web_tool]

from langchain_ollama.llms import OllamaLLM
from langchain_ollama import ChatOllama
from langchain.agents.middleware import dynamic_prompt, ModelRequest
llm=ChatOllama(model="llama3.1")

@dynamic_prompt
def prompt(request: ModelRequest)-> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    docs_content = "\n\n".join(f"{doc.page_content}\nMetadata: {doc.metadata}" for doc in document)
    system_message = (
        """You are a helpful AI assistant. Use the following context to answer the question.
        Think step by step before providing a detailed answer."""
        f"\n\n{docs_content}"
    )
    return system_message



In [29]:
from langchain.agents import create_agent
agent=create_agent(llm,tools=Tools,middleware=[prompt])

In [30]:
response = agent.invoke({"messages": [{"role": "user", "content": "What meaning of MCP"}]})
print("Agent response:", response)

Agent response: {'messages': [HumanMessage(content='What meaning of MCP', additional_kwargs={}, response_metadata={}, id='1c7f7a95-c055-4fa0-a9de-32e4f40b745b'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-02T00:49:41.74554Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2370645083, 'load_duration': 56158375, 'prompt_eval_count': 902, 'prompt_eval_duration': 1757769833, 'eval_count': 20, 'eval_duration': 553686917, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--a62ddcdb-72b9-404c-867c-6f10364411c1-0', tool_calls=[{'name': 'model context protocol', 'args': {'query': 'meaning of MCP'}, 'id': '9d71a793-7720-4ce6-b668-a299c52ab917', 'type': 'tool_call'}], usage_metadata={'input_tokens': 902, 'output_tokens': 20, 'total_tokens': 922}), ToolMessage(content='Think of MCP like a USB-C port for AI applications. Just as USB-C provides a standardized way to connect electronic devices, MCP provides a stand

In [31]:
response = agent.invoke({"messages": [{"role": "user", "content": "Explain about Donald Trump family background"}]})
print("Agent response:", response)

Agent response: {'messages': [HumanMessage(content='Explain about Donald Trump family background', additional_kwargs={}, response_metadata={}, id='c079b01a-a4d8-42c1-991e-4b59fabb5cd1'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-02T00:49:47.610638Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1652622959, 'load_duration': 48717292, 'prompt_eval_count': 904, 'prompt_eval_duration': 1051051334, 'eval_count': 20, 'eval_duration': 551104875, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--52ce980b-b27e-4ae0-ba21-947c97dcc3b2-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Donald Trump family background'}, 'id': 'b6268e05-9700-4f8e-b882-0163588e0015', 'type': 'tool_call'}], usage_metadata={'input_tokens': 904, 'output_tokens': 20, 'total_tokens': 924}), ToolMessage(content="Page: Donald Trump\nSummary: Donald John Trump (born June 14, 1946) is an American politician, media personality, an

In [32]:
response = agent.invoke({
    "messages": [{"role": "user", "content": "Summarize the research papers about Transformer model"}]
})

# Extract tool names and final answer
tool_names = []
final_answer = ""

for msg in response.get("messages", []):
    # Capture tool calls
    if hasattr(msg, "tool_calls") and msg.tool_calls:
        for tool in msg.tool_calls:
            tool_names.append(tool.get("name", "Unknown Tool"))
    # Capture final answer
    if getattr(msg, "content", None) and msg.__class__.__name__ == "AIMessage":
        if msg.content.strip():
            final_answer = msg.content.strip()

# Deduplicate tool names (if multiple calls)
tool_names = list(set(tool_names))

# Clean Display
print("Tools Used:", ", ".join(tool_names) if tool_names else "None")
print("\nAgent Response:\n", final_answer)


Tools Used: arxiv

Agent Response:
 It appears that there are no research papers specifically about the "Transformer model" as requested. However, I can provide a general overview of the Transformer model and its applications.

The Transformer is a neural network architecture introduced in 2017 by Vaswani et al. (1) that revolutionized natural language processing (NLP). It replaced traditional recurrent neural networks (RNNs) and convolutional neural networks (CNNs) with self-attention mechanisms, allowing the model to attend to all positions in a sequence simultaneously.

Since its introduction, the Transformer has been widely adopted in various NLP tasks, such as machine translation, text classification, question answering, and language modeling. It has also been applied to other domains, including computer vision and speech recognition.

There are many variants of the Transformer architecture, each with their own strengths and weaknesses. Some popular variants include:

* BERT (Devl

In [33]:
response = agent.invoke({"messages": [{"role": "user", "content": "Summarize the research papers about Trabsformer model "}]})
print("Agent response:", response)

Agent response: {'messages': [HumanMessage(content='Summarize the research papers about Trabsformer model ', additional_kwargs={}, response_metadata={}, id='88fa6f8a-f590-417b-9d1f-4cf8e24730b2'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-11-02T00:50:33.893908Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1654220209, 'load_duration': 60163667, 'prompt_eval_count': 910, 'prompt_eval_duration': 1043387209, 'eval_count': 20, 'eval_duration': 549344458, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--4295b2c7-a28f-4063-baad-6e8a374aa205-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'Trasformer model'}, 'id': 'f891d314-edf5-46c1-957e-087f9a084159', 'type': 'tool_call'}], usage_metadata={'input_tokens': 910, 'output_tokens': 20, 'total_tokens': 930}), ToolMessage(content="Published: 1997-04-18\nTitle: Renormalization trasformations of the 4D BFYM theory\nAuthors: Alberto Accardi, Andrea Belli\n